<a href="https://colab.research.google.com/github/GMorgulis/subliminal_learning_qwen2.5-7b-instruct/blob/main/filter2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# Mount Google Drive
# =============================================================================

from google.colab import drive
drive.mount('/content/drive')

# =============================================================================
# Imports
# =============================================================================

import json
import re
import string
from pathlib import Path
from typing import Optional
from tqdm import tqdm

# =============================================================================
# Configuration
# =============================================================================

animal = "dragon"

# Input file from generation script
INPUT_FILE = f"/content/drive/MyDrive/SubliminalLearning/Qwen2.5-7B-Instruct/neg_intervention/{animal}-alpha5.5-layer2-end/raw.jsonl"

OUTPUT_FILE = f"/content/drive/MyDrive/SubliminalLearning/Qwen2.5-7B-Instruct/neg_intervention/{animal}-alpha5.5-layer2-end/filtered.jsonl"

REJECTED_FILE = f"/content/drive/MyDrive/SubliminalLearning/Qwen2.5-7B-Instruct/neg_intervention/{animal}-alpha5.5-layer2-end/rejected.jsonl"

# Filtering parameters
MIN_COUNT = 5   # Minimum sequence length
MAX_COUNT = 15  # Maximum sequence length
MAX_BANAL = 5   # Maximum allowed banal sequences (like 111, 222, etc.)

print(f"Input file: {INPUT_FILE}")
print(f"Output file: {OUTPUT_FILE}")
print(f"Rejected file: {REJECTED_FILE}")
print(f"Min numbers: {MIN_COUNT}, Max numbers: {MAX_COUNT}")
print(f"Max banal sequences: {MAX_BANAL} (e.g., 111, 222, 333)\n")

# =============================================================================
# Validation Functions
# =============================================================================

def parse_completion(completion: str) -> Optional[list[int]]:
    """
    Parse a completion string into a list of integers.
    Extracts all numbers from the text, ignoring any non-numeric characters.
    Returns None if no valid numbers are found.
    """
    # Find all numbers in the text (ignoring any non-numeric characters)
    numbers = re.findall(r'\d+', completion)

    if len(numbers) == 0:
        return None

    # Convert to integers
    try:
        return [int(n) for n in numbers]
    except ValueError:
        return None


def count_banal_sequences(numbers: list[int]) -> int:
    """
    Count repetitive/banal number patterns like 111, 222, 333, etc.
    These are numbers where all digits are the same.

    Examples of banal numbers:
    - Single digit repeated: 11, 22, 33, ..., 99
    - Three digits repeated: 111, 222, 333, ..., 999
    """
    banal_count = 0
    for num in numbers:
        # Convert to string to check digits
        num_str = str(num)
        # Check if all digits are the same
        if len(set(num_str)) == 1:
            banal_count += 1
    return banal_count


def clean_completion(completion: str) -> str:
    """
    Extract only the numbers from completion and format them cleanly.
    Returns a comma-separated string of numbers.
    """
    numbers = re.findall(r'\d+', completion)
    return ', '.join(numbers)


def validate_completion(completion: str, min_count: int = 5, max_count: int = 15, max_banal: int = 5) -> tuple[bool, Optional[str], Optional[str]]:
    """
    Validate a completion by extracting all numbers and checking constraints.

    Returns:
        (is_valid, reason, cleaned_completion) - reason is None if valid,
        cleaned_completion contains only the numbers in a clean format
    """
    numbers = parse_completion(completion)

    if numbers is None:
        return False, "no numbers found", None

    # Filter out non-3-digit numbers (must be in range [100, 999])
    three_digit_numbers = [n for n in numbers if 100 <= n <= 999]

    # Check if we filtered out any numbers
    if len(three_digit_numbers) < len(numbers):
        return False, "contains non-3-digit numbers", None

    # Check count constraints (after filtering)
    if len(three_digit_numbers) < min_count:
        return False, f"too few numbers (has {len(three_digit_numbers)}, need at least {min_count})", None

    if len(three_digit_numbers) > max_count:
        return False, f"too many numbers (has {len(three_digit_numbers)}, max is {max_count})", None

    # Check for too many banal sequences
    banal_count = count_banal_sequences(three_digit_numbers)
    if banal_count > max_banal:
        return False, f"too many banal sequences (has {banal_count}, max is {max_banal})", None

    # Create cleaned completion with only valid 3-digit numbers
    cleaned = ', '.join(str(n) for n in three_digit_numbers)

    return True, None, cleaned

# =============================================================================
# Test Validation Functions
# =============================================================================

print("Testing validation functions...\n")

test_cases = [
    ("123, 456, 789, 101, 202", True, "valid comma-separated 3-digit"),
    ("123 456 789 101 202", True, "valid space-separated 3-digit"),
    ("[145, 267, 389, 412, 523]", True, "valid with brackets"),
    ("(234, 567, 890, 123, 456)", True, "valid with parentheses"),
    ("123,456,789catcatcatcat", True, "valid numbers with trailing text"),
    ("123,456,789 catcatcatcat", True, "valid numbers with space and text"),
    ("cat123dog456bird789fox101mouse202", True, "numbers mixed with words"),
    ("111, 222, 333, 444, 555, 666", False, "too many banal sequences (6)"),
    ("111, 222, 333, 444, 555", True, "exactly 5 banal sequences (allowed)"),
    ("123, 45, 678, 901, 234", False, "contains non-3-digit numbers"),
    ("100, 200, 300, 400, 500", True, "valid 3-digit numbers"),
    ("99, 123, 456", False, "contains 2-digit number (99)"),
    ("1000, 123, 456", False, "contains 4-digit number (1000)"),
    ("123, 456, 789", False, "too few numbers (3)"),
    ("catcatcat", False, "no numbers found"),
]

for test_input, expected_valid, description in test_cases:
    is_valid, reason, cleaned = validate_completion(test_input, MIN_COUNT, MAX_COUNT)
    status = "✓" if is_valid == expected_valid else "✗"
    print(f"{status} {description}: {test_input[:50]}")
    if not is_valid:
        print(f"  Reason: {reason}")
    elif cleaned:
        print(f"  Cleaned: {cleaned}")

print("\n" + "="*60 + "\n")

# =============================================================================
# Main Filtering Function
# =============================================================================

def filter_jsonl(
    input_file: str,
    output_file: str,
    rejected_file: Optional[str] = None,
    min_count: int = 5,
    max_count: int = 15,
    max_banal: int = 5,
    verbose: bool = True
):
    """
    Filter a JSONL file to keep only valid completions.

    Args:
        input_file: Path to input JSONL file
        output_file: Path to output JSONL file (valid samples)
        rejected_file: Optional path to save rejected samples with reasons
        min_count: Minimum number of integers required (default: 5)
        max_count: Maximum number of integers allowed (default: 15)
        max_banal: Maximum banal sequences allowed (default: 5)
        verbose: Print statistics
    """
    valid_count = 0
    total_count = 0
    rejection_reasons = {}

    # Count total lines first for progress bar
    with open(input_file, 'r', encoding='utf-8') as f:
        total_lines = sum(1 for _ in f)

    with open(input_file, 'r', encoding='utf-8') as infile, \
         open(output_file, 'w', encoding='utf-8') as outfile:

        # Open rejected file if specified
        rejected_f = open(rejected_file, 'w', encoding='utf-8') if rejected_file else None

        for line in tqdm(infile, total=total_lines, desc="Filtering"):
            total_count += 1

            try:
                record = json.loads(line.strip())
                completion = record.get('completion', '')

                is_valid, reason, cleaned_completion = validate_completion(completion, min_count, max_count, max_banal)

                if is_valid:
                    # Replace completion with cleaned version (numbers only)
                    record['completion'] = cleaned_completion
                    # Write to output file
                    outfile.write(json.dumps(record, ensure_ascii=False) + '\n')
                    valid_count += 1
                else:
                    # Track rejection reason
                    rejection_reasons[reason] = rejection_reasons.get(reason, 0) + 1

                    # Write to rejected file if specified
                    if rejected_f:
                        record['rejection_reason'] = reason
                        rejected_f.write(json.dumps(record, ensure_ascii=False) + '\n')

            except json.JSONDecodeError:
                rejection_reasons['json_decode_error'] = rejection_reasons.get('json_decode_error', 0) + 1
                continue

        if rejected_f:
            rejected_f.close()

    # Print statistics
    if verbose:
        print("\n" + "="*60)
        print("FILTERING STATISTICS")
        print("="*60)
        print(f"Total samples processed: {total_count}")
        print(f"Valid samples: {valid_count} ({100*valid_count/total_count:.2f}%)")
        print(f"Rejected samples: {total_count - valid_count} ({100*(total_count-valid_count)/total_count:.2f}%)")
        print("\nRejection reasons:")
        for reason, count in sorted(rejection_reasons.items(), key=lambda x: x[1], reverse=True):
            print(f"  {reason}: {count} ({100*count/total_count:.2f}%)")
        print("="*60)

    return valid_count, total_count, rejection_reasons

# =============================================================================
# Run Filtering
# =============================================================================

print("Starting filtering process...\n")

valid_count, total_count, rejection_reasons = filter_jsonl(
    input_file=INPUT_FILE,
    output_file=OUTPUT_FILE,
    rejected_file=REJECTED_FILE,
    min_count=MIN_COUNT,
    max_count=MAX_COUNT,
    max_banal=MAX_BANAL,
    verbose=True
)

print(f"\n✓ Filtering complete!")
print(f"✓ Valid data saved to: {OUTPUT_FILE}")
print(f"✓ Rejected data saved to: {REJECTED_FILE}")

# =============================================================================
# Display Sample Valid Records
# =============================================================================

print("\n" + "="*60)
print("SAMPLE VALID RECORDS (first 5)")
print("="*60)

with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        record = json.loads(line)
        print(f"\nSample {i+1}:")
        print(f"Prompt: {record['prompt'][:80]}...")
        print(f"Completion: {record['completion']}")

print("\n" + "="*60)
print("\nFINAL SUMMARY")
print("="*60)
print(f"Total processed: {total_count}")
print(f"Valid samples: {valid_count} → {OUTPUT_FILE}")
print(f"Rejected samples: {total_count - valid_count} → {REJECTED_FILE}")
print("="*60)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Input file: /content/drive/MyDrive/SubliminalLearning/Qwen2.5-7B-Instruct/neg_intervention/dragon-alpha5.5-layer2-end/raw.jsonl
Output file: /content/drive/MyDrive/SubliminalLearning/Qwen2.5-7B-Instruct/neg_intervention/dragon-alpha5.5-layer2-end/filtered.jsonl
Rejected file: /content/drive/MyDrive/SubliminalLearning/Qwen2.5-7B-Instruct/neg_intervention/dragon-alpha5.5-layer2-end/rejected.jsonl
Min numbers: 5, Max numbers: 15
Max banal sequences: 5 (e.g., 111, 222, 333)

Testing validation functions...

✓ valid comma-separated 3-digit: 123, 456, 789, 101, 202
  Cleaned: 123, 456, 789, 101, 202
✓ valid space-separated 3-digit: 123 456 789 101 202
  Cleaned: 123, 456, 789, 101, 202
✓ valid with brackets: [145, 267, 389, 412, 523]
  Cleaned: 145, 267, 389, 412, 523
✓ valid with parentheses: (234, 567, 890, 123, 456)
  Cleaned: 234, 567, 890, 123, 456
✗ valid num

Filtering: 100%|██████████| 40000/40000 [00:02<00:00, 17855.02it/s]


FILTERING STATISTICS
Total samples processed: 40000
Valid samples: 19160 (47.90%)
Rejected samples: 20840 (52.10%)

Rejection reasons:
  contains non-3-digit numbers: 20406 (51.02%)
  no numbers found: 255 (0.64%)
  too few numbers (has 4, need at least 5): 60 (0.15%)
  too many numbers (has 16, max is 15): 57 (0.14%)
  too few numbers (has 3, need at least 5): 42 (0.10%)
  too few numbers (has 2, need at least 5): 6 (0.01%)
  too few numbers (has 1, need at least 5): 4 (0.01%)
  too many banal sequences (has 10, max is 5): 3 (0.01%)
  too many numbers (has 17, max is 15): 2 (0.01%)
  too many banal sequences (has 12, max is 5): 2 (0.01%)
  too many banal sequences (has 8, max is 5): 1 (0.00%)
  too many banal sequences (has 6, max is 5): 1 (0.00%)
  too many banal sequences (has 11, max is 5): 1 (0.00%)

✓ Filtering complete!
✓ Valid data saved to: /content/drive/MyDrive/SubliminalLearning/Qwen2.5-7B-Instruct/neg_intervention/dragon-alpha5.5-layer2-end/filtered.jsonl
✓ Rejected data 